# Question 2 - Part 1: Jays Schedule
#### Obtain, Clean & Prepare a Toronto Blue Jays home games dataset
> This notebook sources Toronto Blue Jays schedule information from https://sports-reference.com. It aggregates schedule information from multiple years, drops irrelevant columns, and engineers new start and end time attributes.

> The resulting dataset is stored as a CSV and is available at ...


In [1]:
def get_game_start_time(date, day_night, year):
  '''
  This function takes as parameters three variables:
  date (str): eg: 'Wednesday, Apr 12'
  day_night (str): 'D' or 'N'
  year (str): eg: '2010'
  
  It returns the start time of a Blue Jays game on that particular day as a Python datetime object
  '''
  
  # gather the start times of games by looking at jays schedule: https://www.mlb.com/bluejays/schedule
  # applies to most home games
  SATURDAY_START_DAYGAME_TIME = '15:07:00'
  SATURDAY_START_NIGHTGAME_TIME = '19:07:00'
  SUNDAY_START_DAYGAME_TIME = '13:07:00'
  SUNDAY_START_NIGHTGAME_TIME = '19:07:00'
  WEEKDAY_START_DAYGAME_TIME = '12:37:00'
  WEEKDAY_START_NIGHTGAME_TIME = '19:07:00'
  
  # format day of week to one of ['SATURDAY', 'SUNDAY', 'WEEKDAY']
  day_of_week = date.split(',')[0]
  if day_of_week in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
    day_of_week = 'WEEKDAY'
  elif day_of_week == 'Saturday':
    day_of_week = 'SATURDAY'
  elif day_of_week == 'Sunday':
    day_of_week = 'SUNDAY'
  else:
    raise NameError('Incorrect Day of Week Detected')
  
  # either a DAYGAME or NIGHTGAME
  game_type = 'DAYGAME'
  if day_night == 'N':
    game_type = 'NIGHTGAME'
    
  # format the start time as string
  start_time = eval('{}_START_{}_TIME'.format(day_of_week, game_type))
  
  # get month and day from string (eg. Apr 21)
  month_and_day = re.findall('[a-zA-Z]{3} \d{1,2}', date)[0]
  
  # produce the time
  datetime_string = '{} {} {}'.format(month_and_day, year, start_time)
 
  # parse it into datetime format
  datetime = parser.parse(datetime_string)
  
  return datetime

In [2]:
def get_game_finish_time(start_datetime, duration):
  '''
  This function takes as parameters two arguments:
  start_datetime (datetime): datetime object representing start of game
  duration (str): eg: '03:30' representing length in hours and minutes of game
  
  returns Python datetime of start_time shifted into future by duration amount
  '''
  
  hours = int(duration.split(':')[0])
  minutes = int(duration.split(':')[1])
  
  delta_duration = relativedelta.relativedelta(hours=+hours, minutes=+minutes)
  
  end_time = start_datetime + delta_duration
  
  return end_time

In [3]:
def read_html(path, year):
  '''
  This function takes two parameters:
  path (str): Path to webpage containing html table of jays schedule
  year (str): eg: '2010'
  
  returns DataFrame of that particular year's Jays Schedule
  '''
  
  dataframe = pd.read_html(path)[0]
  
  # rename the columns
  dataframe.columns = JAYS_SCHEDULE_COLUMNS
  
  # drop all the columns that we don't need
  dataframe.drop(['game_number', 'boxscore_link', 'win_loss', 'innings', 'win_loss', 'rank', 'games_score', 'games_behind', 'win', 'loss', 'save', 'streak', 'originally_scheduled' ], axis=1, inplace=True)
  
  # get all the home games, then drop the home_game column
  dataframe = dataframe[dataframe.home_game != '@'].drop('home_game', axis=1)
  
  # get rid of month separator row inherited from html table
  dataframe = dataframe[dataframe.attendance != 'Attendance']
    
  # set the start time of the game
  dataframe['game_start'] = dataframe.apply(lambda x: get_game_start_time(x.date, x.day_night, year), axis=1)
  
  # drop the date and day_night columns; they're no longer needed
  dataframe.drop(['date', 'day_night'], axis=1, inplace=True)
  
  # convert game duration into timedelta format
  dataframe['game_finish'] = dataframe.apply(lambda x: get_game_finish_time(x.game_start, x.game_duration), axis=1)

  return dataframe

In [4]:
# Question 2
# analyzing the impact of major sporting events of traffic accidents

import pandas as pd
import numpy as np

from string import Template
from dateutil import parser
from dateutil import relativedelta
import re

JAYS_SCHEDULE_PATH_TEMPLATE = Template('https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fteams%2FTOR%2F$date-schedule-scores.shtml&div=div_team_schedule')
JAYS_SCHEDULE_COLUMNS = ['game_number',	'date', 'boxscore_link', 'team', 'home_game','opponent', 'win_loss', 'home_score','away_score', 'innings','games_score','rank','games_behind','win','loss','save','game_duration','day_night','attendance','streak','originally_scheduled']

# we are concerned only with data from 2010 to 2017
START_YEAR = 2008
END_YEAR = 2018

# specify download filename
FILENAME = 'toronto_blue_jays_home_games_scheduled_start_and_finish_times_{}_to_{}.csv'.format(START_YEAR, END_YEAR)

In [5]:
# create a numpy list of years
year_range = np.arange(START_YEAR, END_YEAR+1)

# get the paths to all of the yearly schedules of the jays games
jays_schedule_paths = [JAYS_SCHEDULE_PATH_TEMPLATE.substitute({'date' : year}) for year in year_range]

# create a dict of data frames according to year
# e.g.: {'2010': DataFrame for 2010, '2011': DataFrame for 2011, ...}
baseball_data = {index+START_YEAR : read_html(path, index+START_YEAR) for (index, path) in enumerate(jays_schedule_paths)}

dataframe = pd.DataFrame()
for year in list(year_range):
  year_df = baseball_data[year]
  dataframe = dataframe.append(year_df)
  
# reset the indexes and drop extra index column
dataframe.reset_index(inplace=True)
dataframe.drop('index', axis=1, inplace=True)

# save the file locally
dataframe.to_csv(FILENAME, index=False)